<a href="https://colab.research.google.com/github/lulas98/ETL/blob/main/examen1920v2LV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. The World Bank's international debt data
<p>No es que los humanos solo tengamos deudas para administrar nuestras necesidades. Un país también puede endeudarse para administrar su economía. Por ejemplo, el gasto en infraestructura es un ingrediente costoso requerido para que los ciudadanos de un país lleven una vida cómoda. El Banco Mundial es la organización que proporciona deuda a los países.</p>

<!-- <p>En este notebook, vamos a analizar los datos de la deuda internacional recopilados por el Banco Mundial. El conjunto de datos contiene información sobre el monto de la deuda (en USD) que deben los países en desarrollo en varias categorías.</p>  -->
    

"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD
"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD
"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD
"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD
"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD
"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD
"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD
"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD
"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD
"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD
"PPG, official creditors (INT, current US$)",DT.INT.OFFT.CD
"Principal repayments on external debt, long-term (AMT, current US$)",DT.AMT.DLXF.CD
"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",DT.INT.DPNG.CD
"PPG, bonds (AMT, current US$)",DT.AMT.PBND.CD
"PPG, bonds (INT, current US$)",DT.INT.PBND.CD
"PPG, commercial banks (AMT, current US$)",DT.AMT.PCBK.CD
"PPG, commercial banks (DIS, current US$)",DT.DIS.PCBK.CD
"PPG, commercial banks (INT, current US$)",DT.INT.PCBK.CD
"PPG, other private creditors (AMT, current US$)",DT.AMT.PROP.CD
"PPG, other private creditors (DIS, current US$)",DT.DIS.PROP.CD
"PPG, other private creditors (INT, current US$)",DT.INT.PROP.CD
"PPG, private creditors (AMT, current US$)",DT.AMT.PRVT.CD
"PPG, private creditors (DIS, current US$)",DT.DIS.PRVT.CD
"PPG, private creditors (INT, current US$)",DT.INT.PRVT.CD
"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",DT.AMT.DPNG.CD

<p>Vamos a encontrar las respuestas a preguntas como:

<p>¿Cuál es el monto total de la deuda que deben los países enumerados en el conjunto de datos?
<p>¿Qué país posee la cantidad máxima de deuda y cómo se ve esa cantidad?
<p>¿Cuál es el monto promedio de la deuda de los países a través de diferentes indicadores de deuda?
    
Además tenemos otro dataset en el que encontramos información histórica de algunos índices de desarrollo, entre los que se encuentran algunos de deuda como son:

Series Name,Series Code
"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN

"Central government debt, total (current LCU)",GC.DOD.TOTL.CN

"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS

#### 1. Inicializar y cargar el contexto spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

NameError: ignored

##### 2. Número de países distintos en cada dataset. Coinciden?

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import Row

Importamos csv

In [ ]:
explicacion_var = sc.textFile('explicacion_var1920.csv')
explicacion_varDeu = sc.textFile('explicacion_varDeu1920.csv')
indicadores_desarrollo = sc.textFile('indicadores_desarrollo.csv')
indicadores_deuda = sc.textFile('indicadores_deuda.csv')

Los observamos

In [ ]:
explicacion_var.take(5)



['Series Name,Series Code',
 '"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN',
 '"Cause of death, by injury (% of total)",SH.DTH.INJR.ZS',
 '"Central government debt, total (current LCU)",GC.DOD.TOTL.CN',
 '"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS']

In [ ]:
explicacion_varDeu.take(5)

['indicator_name,indicator_code',
 '"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD',
 '"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD',
 '"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD',
 '"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD']

In [ ]:
indicadores_desarrollo.take(5)

['country_name,country_code,anho,GC.DOD.TOTL.CN,GC.DOD.TOTL.GD.ZS,SP.DYN.CBRT.IN',
 'Albania,ALB,2014 [YR2014],1023040800000,73.3202469198429,12.259',
 'Albania,ALB,2015 [YR2015],1145500940000,79.8644448747259,12.197',
 'Albania,ALB,2016 [YR2016],1188814490000,80.7355748396923,12.08',
 'Australia,AUS,2014 [YR2014],674700000,0.042207528166503,13.2']

In [ ]:
indicadores_deuda.take(5)

['country_name,country_code,DT.AMT.BLAT.CD,DT.AMT.DLXF.CD,DT.AMT.DPNG.CD,DT.AMT.MLAT.CD,DT.AMT.OFFT.CD,DT.AMT.PBND.CD,DT.AMT.PCBK.CD,DT.AMT.PROP.CD,DT.AMT.PRVT.CD,DT.DIS.BLAT.CD,DT.DIS.DLXF.CD,DT.DIS.MLAT.CD,DT.DIS.OFFT.CD,DT.DIS.PCBK.CD,DT.DIS.PROP.CD,DT.DIS.PRVT.CD,DT.INT.BLAT.CD,DT.INT.DLXF.CD,DT.INT.DPNG.CD,DT.INT.MLAT.CD,DT.INT.OFFT.CD,DT.INT.PBND.CD,DT.INT.PCBK.CD,DT.INT.PROP.CD,DT.INT.PRVT.CD',
 'Afghanistan,AFG,61739336.9,100847181.9,,39107845,100847181.9,,,,,49114729.4,72894453.7,23779724.3,72894453.7,,,,39903620.1,53239440.1,,13335820,53239440.1,,,,',
 'Albania,ALB,54250280.6,790248675.2,514185620,182197616.7,236447897.3,0,39445139.5,170018.4,39615157.9,76050616.1,317194512.5,234321242.3,310371858.4,4542664.9,2279989.2,6822654.1,13847333.6,165602386.9,87884000,28101536.1,41948869.7,31030688.2,4618504.3,120324.7,35769517.2',
 'Algeria,DZA,95188724.6,171185188.1,75420000,0,95188724.6,,576463.5,0,576463.5,10320772.2,10320772.2,,10320772.2,,,,7680627.6,19031728.7,3220000,8094779,


Eliminamos los cabeceros

In [ ]:

cabecero = explicacion_var.first()
explicacion_var_clean = explicacion_var.filter(lambda x: x !=cabecero)
explicacion_var_clean.take(5)

['"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN',
 '"Cause of death, by injury (% of total)",SH.DTH.INJR.ZS',
 '"Central government debt, total (current LCU)",GC.DOD.TOTL.CN',
 '"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS',
 '"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",SH.ALC.PCAP.LI']

In [ ]:
cabecero2 = indicadores_desarrollo.first()
indicadores_desarrollo_clean = indicadores_desarrollo.filter(lambda x: x !=cabecero2)
indicadores_desarrollo_clean.take(5)

['Albania,ALB,2014 [YR2014],1023040800000,73.3202469198429,12.259',
 'Albania,ALB,2015 [YR2015],1145500940000,79.8644448747259,12.197',
 'Albania,ALB,2016 [YR2016],1188814490000,80.7355748396923,12.08',
 'Australia,AUS,2014 [YR2014],674700000,0.042207528166503,13.2',
 'Australia,AUS,2015 [YR2015],762718000,0.0469480198522591,12.9']

In [ ]:
cabecero3 = explicacion_varDeu.first()
explicacion_varDeu_clean = explicacion_varDeu.filter(lambda x: x !=cabecero3)
explicacion_varDeu_clean.take(5)

['"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD',
 '"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD',
 '"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD',
 '"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD',
 '"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD']

In [ ]:
indicadores_deuda.take(5)

['country_name,country_code,DT.AMT.BLAT.CD,DT.AMT.DLXF.CD,DT.AMT.DPNG.CD,DT.AMT.MLAT.CD,DT.AMT.OFFT.CD,DT.AMT.PBND.CD,DT.AMT.PCBK.CD,DT.AMT.PROP.CD,DT.AMT.PRVT.CD,DT.DIS.BLAT.CD,DT.DIS.DLXF.CD,DT.DIS.MLAT.CD,DT.DIS.OFFT.CD,DT.DIS.PCBK.CD,DT.DIS.PROP.CD,DT.DIS.PRVT.CD,DT.INT.BLAT.CD,DT.INT.DLXF.CD,DT.INT.DPNG.CD,DT.INT.MLAT.CD,DT.INT.OFFT.CD,DT.INT.PBND.CD,DT.INT.PCBK.CD,DT.INT.PROP.CD,DT.INT.PRVT.CD',
 'Afghanistan,AFG,61739336.9,100847181.9,,39107845,100847181.9,,,,,49114729.4,72894453.7,23779724.3,72894453.7,,,,39903620.1,53239440.1,,13335820,53239440.1,,,,',
 'Albania,ALB,54250280.6,790248675.2,514185620,182197616.7,236447897.3,0,39445139.5,170018.4,39615157.9,76050616.1,317194512.5,234321242.3,310371858.4,4542664.9,2279989.2,6822654.1,13847333.6,165602386.9,87884000,28101536.1,41948869.7,31030688.2,4618504.3,120324.7,35769517.2',
 'Algeria,DZA,95188724.6,171185188.1,75420000,0,95188724.6,,576463.5,0,576463.5,10320772.2,10320772.2,,10320772.2,,,,7680627.6,19031728.7,3220000,8094779,

In [ ]:

cabecero4 = indicadores_deuda.first()
indicadores_deuda_clean = indicadores_deuda.filter(lambda x: x !=cabecero4)
indicadores_deuda_clean.take(5)

['Afghanistan,AFG,61739336.9,100847181.9,,39107845,100847181.9,,,,,49114729.4,72894453.7,23779724.3,72894453.7,,,,39903620.1,53239440.1,,13335820,53239440.1,,,,',
 'Albania,ALB,54250280.6,790248675.2,514185620,182197616.7,236447897.3,0,39445139.5,170018.4,39615157.9,76050616.1,317194512.5,234321242.3,310371858.4,4542664.9,2279989.2,6822654.1,13847333.6,165602386.9,87884000,28101536.1,41948869.7,31030688.2,4618504.3,120324.7,35769517.2',
 'Algeria,DZA,95188724.6,171185188.1,75420000,0,95188724.6,,576463.5,0,576463.5,10320772.2,10320772.2,,10320772.2,,,,7680627.6,19031728.7,3220000,8094779,15775406.6,,13192.3,23129.8,36322.1',
 'Angola,AGO,8473824016.3,11067045628.1,,124688691.9,8598512708.2,1e+09,1125088719.9,343444200,2468532919.9,8838256901.1,10924018093.1,178989598.2,9017246499.3,1906771593.8,,1906771593.8,1005053965.1,1798550445.5,,47038476.1,1052092441.2,7e+07,580902004.3,95556000,746458004.3',
 'Armenia,ARM,68968314.7,845630723.8,680696190,94331207.1,163299521.8,0,1635012,,1635012

In [ ]:


indicadores_desarrollo_clean = indicadores_desarrollo_clean.map(lambda x: x.split(','))



In [ ]:
indicadores_desarrollo_clean.take(5)

[['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259'],
 ['Albania',
  'ALB',
  '2015 [YR2015]',
  '1145500940000',
  '79.8644448747259',
  '12.197'],
 ['Albania',
  'ALB',
  '2016 [YR2016]',
  '1188814490000',
  '80.7355748396923',
  '12.08'],
 ['Australia',
  'AUS',
  '2014 [YR2014]',
  '674700000',
  '0.042207528166503',
  '13.2'],
 ['Australia',
  'AUS',
  '2015 [YR2015]',
  '762718000',
  '0.0469480198522591',
  '12.9']]

In [ ]:
indicadores_deuda_clean = indicadores_deuda_clean.map(lambda x: x.split(','))

In [ ]:
indicadores_deuda_clean.take(5)

[['Afghanistan',
  'AFG',
  '61739336.9',
  '100847181.9',
  '',
  '39107845',
  '100847181.9',
  '',
  '',
  '',
  '',
  '49114729.4',
  '72894453.7',
  '23779724.3',
  '72894453.7',
  '',
  '',
  '',
  '39903620.1',
  '53239440.1',
  '',
  '13335820',
  '53239440.1',
  '',
  '',
  '',
  ''],
 ['Albania',
  'ALB',
  '54250280.6',
  '790248675.2',
  '514185620',
  '182197616.7',
  '236447897.3',
  '0',
  '39445139.5',
  '170018.4',
  '39615157.9',
  '76050616.1',
  '317194512.5',
  '234321242.3',
  '310371858.4',
  '4542664.9',
  '2279989.2',
  '6822654.1',
  '13847333.6',
  '165602386.9',
  '87884000',
  '28101536.1',
  '41948869.7',
  '31030688.2',
  '4618504.3',
  '120324.7',
  '35769517.2'],
 ['Algeria',
  'DZA',
  '95188724.6',
  '171185188.1',
  '75420000',
  '0',
  '95188724.6',
  '',
  '576463.5',
  '0',
  '576463.5',
  '10320772.2',
  '10320772.2',
  '',
  '10320772.2',
  '',
  '',
  '',
  '7680627.6',
  '19031728.7',
  '3220000',
  '8094779',
  '15775406.6',
  '',
  '13192.3'

Contamos el numero de paises distintos

In [ ]:
desarrollo_countries = indicadores_desarrollo_clean.map(lambda x: x[0]) #es la posición 0 de nuestro RDD que en este caso son los nombres de los paises
desarrollo_countries.distinct().count()

51

In [ ]:

indicadores_deuda_countries = indicadores_deuda_clean.map(lambda x: x[0])
indicadores_deuda_countries.distinct().count()

123

Se puede observar que son distintos

##### 3. Total de deuda contraida por cada pais: total amount of debt (in USD) DT.AMT.MLAT.CD

In [ ]:

indicadores_deuda_split = indicadores_deuda.map(lambda x: x.split(','))
indicadores_deuda_split.take(1)

[['country_name',
  'country_code',
  'DT.AMT.BLAT.CD',
  'DT.AMT.DLXF.CD',
  'DT.AMT.DPNG.CD',
  'DT.AMT.MLAT.CD',
  'DT.AMT.OFFT.CD',
  'DT.AMT.PBND.CD',
  'DT.AMT.PCBK.CD',
  'DT.AMT.PROP.CD',
  'DT.AMT.PRVT.CD',
  'DT.DIS.BLAT.CD',
  'DT.DIS.DLXF.CD',
  'DT.DIS.MLAT.CD',
  'DT.DIS.OFFT.CD',
  'DT.DIS.PCBK.CD',
  'DT.DIS.PROP.CD',
  'DT.DIS.PRVT.CD',
  'DT.INT.BLAT.CD',
  'DT.INT.DLXF.CD',
  'DT.INT.DPNG.CD',
  'DT.INT.MLAT.CD',
  'DT.INT.OFFT.CD',
  'DT.INT.PBND.CD',
  'DT.INT.PCBK.CD',
  'DT.INT.PROP.CD',
  'DT.INT.PRVT.CD']]

In [ ]:

country_deuda = indicadores_deuda_clean.map(lambda x: (x[0], x[5])) # Podemos observar que la deuda (DT.AMT.MLAT.CD) esta en la posicion 6, por lo que cogemos x[5]
country_deuda.distinct().take(123) #Lista con los 123 paises distintos y su deuda

[('Afghanistan', '39107845'),
 ('Albania', '182197616.7'),
 ('Algeria', '0'),
 ('Armenia', '94331207.1'),
 ('Belarus', '595438826'),
 ('Bosnia and Herzegovina', '290170946.5'),
 ('Brazil', '2514318741.6'),
 ('Bulgaria', '228993019.8'),
 ('Burundi', '16115111.8'),
 ('Cameroon', '152095968.1'),
 ('Central African Republic', '2241813.8'),
 ('Chad', '95738709.5'),
 ('Comoros', '2182053.1'),
 ('Costa Rica', '272808260.7'),
 ("Cote d'Ivoire", '84629406.5'),
 ('Dominica', '11130355.8'),
 ('Dominican Republic', '254728306.1'),
 ('Ecuador', '578894793.9'),
 ('"Egypt', '94662397.3'),
 ('El Salvador', '298939687.1'),
 ('"Gambia', ''),
 ('Georgia', '144062401.5'),
 ('Grenada', '13740888.8'),
 ('Guatemala', '339822253.6'),
 ('Guinea-Bissau', '6713257.2'),
 ('Guyana', '34651806.5'),
 ('India', '4545609909.9'),
 ('Indonesia', '2469145824.7'),
 ('Jordan', '270498715.5'),
 ('Kosovo', '33590154.6'),
 ('Lesotho', '32459403.7'),
 ('Liberia', '7383619.4'),
 ('"Macedonia', '571657452.1'),
 ('Malawi', '35659

##### 4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD

In [ ]:
indicadores_deuda_split = indicadores_deuda.map(lambda x: x.split(','))
indicadores_deuda_split.take(1) # Volvemos a observar que los indicadores requeridos están en las posiciones 3, 12 y 19

[['country_name',
  'country_code',
  'DT.AMT.BLAT.CD',
  'DT.AMT.DLXF.CD',
  'DT.AMT.DPNG.CD',
  'DT.AMT.MLAT.CD',
  'DT.AMT.OFFT.CD',
  'DT.AMT.PBND.CD',
  'DT.AMT.PCBK.CD',
  'DT.AMT.PROP.CD',
  'DT.AMT.PRVT.CD',
  'DT.DIS.BLAT.CD',
  'DT.DIS.DLXF.CD',
  'DT.DIS.MLAT.CD',
  'DT.DIS.OFFT.CD',
  'DT.DIS.PCBK.CD',
  'DT.DIS.PROP.CD',
  'DT.DIS.PRVT.CD',
  'DT.INT.BLAT.CD',
  'DT.INT.DLXF.CD',
  'DT.INT.DPNG.CD',
  'DT.INT.MLAT.CD',
  'DT.INT.OFFT.CD',
  'DT.INT.PBND.CD',
  'DT.INT.PCBK.CD',
  'DT.INT.PROP.CD',
  'DT.INT.PRVT.CD']]

In [ ]:
indicador_deuda1 = indicadores_deuda_clean.map(lambda x: x[2])

indicador_deuda1.take(3)


['61739336.9', '54250280.6', '95188724.6']

In [ ]:
indicador_deuda2 = indicadores_deuda_clean.map(lambda x: x[11])
indicador_deuda2.take(3)

['49114729.4', '76050616.1', '10320772.2']

In [ ]:
indicador_deuda3 = indicadores_deuda_clean.map(lambda x: x[18])
indicador_deuda3.take(3)

['39903620.1', '13847333.6', '7680627.6']

In [ ]:
indicador_deuda1
indicador_deuda1 = indicador_deuda1.filter(lambda x: x.isdigit())
indicador_deuda1 = indicador_deuda1.map(lambda x: float(x))

indicador_deuda1.mean() #DT.AMT.BLAT.CD

61416650.11111111

In [ ]:
indicador_deuda2
indicador_deuda2 = indicador_deuda2.filter(lambda x: x.isdigit())
indicador_deuda2 = indicador_deuda2.map(lambda x: float(x))

indicador_deuda2.mean() #DT.DIS.BLAT.CD

506624979.33333343

In [ ]:
indicador_deuda3
indicador_deuda3 = indicador_deuda3.filter(lambda x: x.isdigit())
indicador_deuda3 = indicador_deuda3.map(lambda x: float(x))
indicador_deuda3.mean() #DT.INT.BLAT.CD

53762351.333333336

In [ ]:
media_total = (indicador_deuda1.mean() + indicador_deuda2.mean() +indicador_deuda3.mean())/3
media_total # Media de las 3

207267993.59259263

##### 5. Los 20 paises con DT.AMT.DLXF.CD más alto

In [ ]:
indicador_deuda4 = indicadores_deuda_clean.map(lambda x: (x[3], x[0])) # Hay que coger primero el numero y despues el nombre del pais porque la funcion top coge la primera columna, y lo que buscamos saber es el top 20 de DT.AMT.DLXF.CD


In [ ]:
indicador_deuda4.top(20)

[('979241934.4', "Cote d'Ivoire"),
 ('96218620835.7', 'China'),
 ('9506919669.6', 'Lebanon'),
 ('9474257551.9', 'South Africa'),
 ('90041840304.1', 'Brazil'),
 ('8873505909.2', 'Vietnam'),
 ('845630723.8', 'Armenia'),
 ('8336013891.3', 'Pakistan'),
 ('83185188.3', 'Djibouti'),
 ('825000', 'Timor-Leste'),
 ('816419045.1', 'Kyrgyz Republic'),
 ('8148995625.6', 'Ukraine'),
 ('790248675.2', 'Albania'),
 ('775955981.3', 'Cambodia'),
 ('769232867', 'Moldova'),
 ('73812591.8', 'Togo'),
 ('70570550.3', '"Macedonia'),
 ('6949184320.5', '"Egypt'),
 ('6783629.1', 'Guinea-Bissau'),
 ('67728445.9', 'Cabo Verde')]

##### 6. Pais con los datos informados todos los años.

In [ ]:
indicadores_desarrollo_clean.take(5)

[['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259'],
 ['Albania',
  'ALB',
  '2015 [YR2015]',
  '1145500940000',
  '79.8644448747259',
  '12.197'],
 ['Albania',
  'ALB',
  '2016 [YR2016]',
  '1188814490000',
  '80.7355748396923',
  '12.08'],
 ['Australia',
  'AUS',
  '2014 [YR2014]',
  '674700000',
  '0.042207528166503',
  '13.2'],
 ['Australia',
  'AUS',
  '2015 [YR2015]',
  '762718000',
  '0.0469480198522591',
  '12.9']]

In [ ]:
number_of_countries = indicadores_desarrollo_clean.map(lambda x: (x[0],1)) # asi conseguimos sumar 1 cada vez que aparezca el pais en el dataset
number_of_years = number_of_countries.countByKey()
number_of_years #Podemos observar que Malawi es el unico que tiene 4 años

defaultdict(int,
            {'"Bahamas': 3,
             '"Korea': 3,
             '"Micronesia': 3,
             'Albania': 3,
             'Australia': 3,
             'Barbados': 2,
             'Belarus': 3,
             'Belize': 1,
             'Bhutan': 3,
             'Botswana': 3,
             'Brazil': 3,
             'Colombia': 3,
             'El Salvador': 3,
             'Georgia': 3,
             'Guatemala': 3,
             'Hungary': 3,
             'Indonesia': 3,
             'Ireland': 3,
             'Jamaica': 3,
             'Japan': 3,
             'Jordan': 2,
             'Kazakhstan': 3,
             'Kyrgyz Republic': 3,
             'Malawi': 4,
             'Malaysia': 3,
             'Moldova': 3,
             'Namibia': 1,
             'New Zealand': 3,
             'Palau': 3,
             'Papua New Guinea': 2,
             'Peru': 3,
             'Philippines': 1,
             'Russian Federation': 3,
             'San Marino': 2,
             'Sey

##### 7. Media anual de los distintos indicadores de desarrollo

In [ ]:
indicadores_desarrollo_clean.take(5)

[['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259'],
 ['Albania',
  'ALB',
  '2015 [YR2015]',
  '1145500940000',
  '79.8644448747259',
  '12.197'],
 ['Albania',
  'ALB',
  '2016 [YR2016]',
  '1188814490000',
  '80.7355748396923',
  '12.08'],
 ['Australia',
  'AUS',
  '2014 [YR2014]',
  '674700000',
  '0.042207528166503',
  '13.2'],
 ['Australia',
  'AUS',
  '2015 [YR2015]',
  '762718000',
  '0.0469480198522591',
  '12.9']]

Existen datos que tienen 7 columnas en lugar de 6, como son todos aquellos paises que tienen un "The" delante como Bahamas, un "Rep." como es el caso de Korea o un "Fed. Sts." como es Micronesia. Esto nos da errores a la hora de calcular la media por columnas, es por ello que vamos a separarlos en otro RDD,eliminar esa columna extra y volver a introducirlos una vez esten formateados.


In [ ]:
dist_format_data = indicadores_desarrollo_clean.filter(lambda x: (len(x) != 6))


In [ ]:
dist_format_data.take(10)

[['"Bahamas',
  ' The"',
  'BHS',
  '2014 [YR2014]',
  '5158420000',
  '47.2672793747079',
  '14.34'],
 ['"Bahamas',
  ' The"',
  'BHS',
  '2015 [YR2015]',
  '5655150000',
  '48.1195170307089',
  '14.224'],
 ['"Bahamas',
  ' The"',
  'BHS',
  '2016 [YR2016]',
  '6138814000',
  '51.4207431481606',
  '14.131'],
 ['"Korea',
  ' Rep."',
  'KOR',
  '2014 [YR2014]',
  '570473174969116',
  '38.3878017121372',
  '8.6'],
 ['"Korea',
  ' Rep."',
  'KOR',
  '2015 [YR2015]',
  '621640877442310',
  '39.7437106767763',
  '8.6'],
 ['"Korea',
  ' Rep."',
  'KOR',
  '2016 [YR2016]',
  '662636705078292',
  '40.3607233268095',
  '7.9'],
 ['"Micronesia',
  ' Fed. Sts."',
  'FSM',
  '2014 [YR2014]',
  '74676687',
  '23.3897380060743',
  '23.365'],
 ['"Micronesia',
  ' Fed. Sts."',
  'FSM',
  '2015 [YR2015]',
  '94767029',
  '29.9431402956919',
  '23.213'],
 ['"Micronesia',
  ' Fed. Sts."',
  'FSM',
  '2016 [YR2016]',
  '84368040',
  '25.3917802774832',
  '23.069']]

In [ ]:
dist_format_data = dist_format_data.map(lambda x: (x[0],x[2],x[3],x[4],x[5],x[6])) # Eliminamos la columna del Prefijo, que daba problemas
desarrollo_limpio = indicadores_desarrollo_clean.filter(lambda x: (len(x)== 6))
desarrollo_limpio = desarrollo_limpio.union(dist_format_data) # Aqui ya tenemos el Rdd limpio y con el mismo formato

In [ ]:
desarrollo_limpio.take(3)


[['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259'],
 ['Albania',
  'ALB',
  '2015 [YR2015]',
  '1145500940000',
  '79.8644448747259',
  '12.197'],
 ['Albania',
  'ALB',
  '2016 [YR2016]',
  '1188814490000',
  '80.7355748396923',
  '12.08']]

# GC.DOD.TOTL.CN

Diferenciamos por años

2014

In [ ]:

desarrollo_limpio2014 = desarrollo_limpio.filter(lambda x: x[2] == '2014 [YR2014]')
desarrollo_limpio2014 = desarrollo_limpio2014.map(lambda x: x[3])
desarrollo_limpio2014 = desarrollo_limpio2014.filter(lambda x: x.isdigit())
desarrollo_limpio2014 = desarrollo_limpio2014.map(lambda x: int(x))
desarrollo_limpio2014.mean()

124490773241364.06

2015


In [ ]:
desarrollo_limpio2015 = desarrollo_limpio.filter(lambda x: x[2] == '2015 [YR2015]')
desarrollo_limpio2015 = desarrollo_limpio2015.map(lambda x: x[3])
desarrollo_limpio2015 = desarrollo_limpio2015.filter(lambda x: x.isdigit())
desarrollo_limpio2015 = desarrollo_limpio2015.map(lambda x: int(x))
desarrollo_limpio2015.mean()

187880226434130.88

2016

In [ ]:
desarrollo_limpio2016 = desarrollo_limpio.filter(lambda x: x[2] == '2016 [YR2016]')
desarrollo_limpio2016 = desarrollo_limpio2016.map(lambda x: x[3])
desarrollo_limpio2016 = desarrollo_limpio2016.filter(lambda x: x.isdigit())
desarrollo_limpio2016 = desarrollo_limpio2016.map(lambda x: int(x))
desarrollo_limpio2016.mean()

266349830024277.47

# GC.DOD.TOTL.GD.ZS

2014

In [ ]:
desarrollo_limpio2014 = desarrollo_limpio.filter(lambda x: x[2] == '2014 [YR2014]')
desarrollo_limpio2014 = desarrollo_limpio2014.map(lambda x: x[4])
desarrollo_limpio2014 = desarrollo_limpio2016.filter(lambda x: x.isdigit())

desarrollo_limpio2014.mean()

Py4JJavaError: ignored

2015

In [ ]:
desarrollo_limpio2015 = desarrollo_limpio.filter(lambda x: x[2] == '2015 [YR2015]')
desarrollo_limpio2015 = desarrollo_limpio2015.map(lambda x: x[4])

desarrollo_limpio2015 = desarrollo_limpio2015.map(lambda x: int(x))
desarrollo_limpio2015.mean()

2016

In [ ]:
desarrollo_limpio2016 = desarrollo_limpio.filter(lambda x: x[2] == '2016 [YR2016]')
desarrollo_limpio2016 = desarrollo_limpio2016.map(lambda x: x[4])

desarrollo_limpio2016 = desarrollo_limpio2016.map(lambda x: int(x))
desarrollo_limpio2016.mean()

**SP.DYN.CBRT.IN**

2014

In [ ]:
desarrollo_limpio2014 = desarrollo_limpio.filter(lambda x: x[2] == '2014 [YR2014]')
desarrollo_limpio2014 = desarrollo_limpio2014.map(lambda x: x[5])
desarrollo_limpio2014 = desarrollo_limpio2014.filter(lambda x: x.isdigit())
desarrollo_limpio2014 = desarrollo_limpio2014.map(lambda x: int(x))
desarrollo_limpio2014.mean()

2015

In [ ]:
desarrollo_limpio2015 = desarrollo_limpio.filter(lambda x: x[2] == '2015 [YR2015]')
desarrollo_limpio2015 = desarrollo_limpio2015.map(lambda x: x[5])
desarrollo_limpio2015 = desarrollo_limpio2015.filter(lambda x: x.isdigit())
desarrollo_limpio2015 = desarrollo_limpio2015.map(lambda x: int(x))
desarrollo_limpio2015.mean()

2016

In [ ]:
desarrollo_limpio2016 = desarrollo_limpio.filter(lambda x: x[2] == '2016 [YR2016]')
desarrollo_limpio2016 = desarrollo_limpio2016.map(lambda x: x[5])
desarrollo_limpio2016 = desarrollo_limpio2016.filter(lambda x: x.isdigit())
desarrollo_limpio2016 = desarrollo_limpio2016.map(lambda x: int(x))
desarrollo_limpio2016.mean()

##### 8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN

In [ ]:
indicadores_deuda.take(2)

In [ ]:
indicadores_desarrollo.take(3)

Calculamos la deuda media de SP.DYN.CBRT.IN

In [ ]:
deuda = indicadores_desarrollo_clean.map(lambda x: x[5])
deuda.mean()


Seleccionamos los datos menores a la media de SP.DYN.CBRT.IN

In [ ]:
indicadores_desarrollo_clean.filter(lambda x: x[5]< deuda.mean())

##### 9. ¿Hay alguna relación entre los nacimientos y el indicador DT.AMT.DLXF.CD? ¿Cómo podrías demostrarlo?